# Provisioning OpenRouter API keys

This notebook is for instructors to provision OpenRouter API keys for workshop participants. It is not intended for use by participants.

However intermediate and advanced participants are welcome to see the code and use it as a reference whenever they need to bulk provision API keys for their own projects.

## OpenRouter API provisioning

OpenRouter provides a simple API to create and manage API keys. 

The core idea is to use a special API key that has the `admin` role, which allows you to create new API keys for other users. This key does not make regular API calls but is used to manage API keys.

This type of key is extremely powerful and should be kept secret. It is not intended for use in production applications or by end users.

If this type of key is compromised, it can be used to create new API keys for any user, which can then be used to make API calls and drain your account's balance to zero.

In [18]:
# first let's load needed libraries and also read system environment variables
# let's load Python sys to see what Python version we are using
import sys
print(f"Python version: {sys.version}")
# now datetime
from datetime import datetime
print(f"Current date and time: {datetime.now()}")
# we will need json to read and write JSON files
import json
# and os to read environment variables
import os

# we also need Pathlib to work with file paths
from pathlib import Path

# now let's read environment variables

# OPEN_ROUTER_BSSDH_PROVISIONER
open_router_bssdh_provisioner = os.getenv("OPEN_ROUTER_BSSDH_PROVISIONER")
if open_router_bssdh_provisioner:
    print("OPEN_ROUTER_BSSDH_PROVISIONER key loaded from environment variables.")
    # README! we do not want to print the key itself for security reasons it is very easy to print it and publish it by mistake
else:
    print("OPEN_ROUTER_BSSDH_PROVISIONER key not found in environment variables. Please set it before running the script.")

# import tqdm
try:
    from tqdm import tqdm
    from tqdm import __version__ as tqdm_version
    print(f"tqdm library is installed: {tqdm_version}")
    # tqdm is used for progress bars in loops
except ImportError:
    print("tqdm library is not installed. Please install it using 'pip install tqdm' from command line terminal.")

# we will need some external libraries, so let's check if they are installed
# first requests
try:
    import requests
    print(f"requests library is installed: {requests.__version__}")
except ImportError:
    print("requests library is not installed. Please install it using 'pip install requests' from command line terminal.")

# we will need Pandas to work with DataFrames
try:
    import pandas as pd
    print(f"Pandas library is installed: {pd.__version__}")
except ImportError:
    print("Pandas library is not installed. Please install it using 'pip install pandas[excel]' from command line terminal.")

Python version: 3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]
Current date and time: 2025-08-06 11:44:31.000753
OPEN_ROUTER_BSSDH_PROVISIONER key loaded from environment variables.
tqdm library is installed: 4.67.1
requests library is installed: 2.32.4
Pandas library is installed: 2.3.1


## Loading Dataframe with workshop participants

Our workshop participants are listed in a XLSX file. From the organizer we hear that some participants have multiple e-mails so we will need to consider that when creating API keys.

First step is reading XLSX file with participants' data and creating a DataFrame with their e-mails, names and other information.



In [6]:
# we store our participant xlsx in a temp folder as this information is sensitive and not meant for public access
# so we will use Pathlib to create a path to the file
temp_folder = Path("../temp")
# list xlsx files in the temp folder
xlsx_files = list(temp_folder.glob("*.xlsx"))
if xlsx_files:
    print(f"Found {len(xlsx_files)} xlsx file(s) in the temp folder: {', '.join([str(file) for file in xlsx_files])}")
    df = pd.read_excel(xlsx_files[0])
    print(f"DataFrame created with {len(df)} rows and {len(df.columns)} columns.")
else:
    print("No xlsx files found in the temp folder. Please make sure to place the file there before running the script.")

Found 1 xlsx file(s) in the temp folder: ..\temp\BSSDH_2025_participants.xlsx
DataFrame created with 54 rows and 3 columns.


In [9]:
# let's see head and tail of the DataFrame
# head and tail of 2 has our instructor's names and e-mails rest are students and workshop participants
# we do not want to print the personal information of the participants, so we will just display the first and last 2 rows
print("DataFrame head:")
display(df.head(2))
print("DataFrame tail:")
display(df.tail(2))

DataFrame head:


,E-mail,Name,Surname
0,anda.baklane@gmail.com,Anda,Baklāne
1,haralds.matulis@gmail.com,Haralds,Matulis


DataFrame tail:


,E-mail,Name,Surname
52,"valdis.saulespurens@gmail.com, valdis.saulespu...",Valdis,Saulespurēns
53,viesturs.veveris@lnb.lv,Viesturs,Vēveris


Now that we have successfully loaded the DataFrame, we can proceed to the next steps of provisioning API keys for each participant. The idea is to create a new column in the dataframe that will store the API keys for each participant. We will use the OpenRouter Provisioning API to create these keys.

## Provisioning API keys

### Creating function to create API keys

First we need to create a function that will use the OpenRouter API to create new API keys. This function name, label and limit with some defaults that can be overridden by the caller.


In [11]:
# now that everything is loaded, let's define a function to provision API keys
def provision_open_router_keys(provision_key=open_router_bssdh_provisioner, 
                               name="Test Customer Instance Key",
                               label="TestCustomer123",
                               limit=1): # 1 USD dollar limit by default
    # we use example from https://openrouter.ai/docs/features/provisioning-api-keys
    PROVISIONING_API_KEY = provision_key
    BASE_URL = "https://openrouter.ai/api/v1/keys"
    # Create a new API key
    response = requests.post(
        # f"{BASE_URL}/", # Documentation for this is WRONG! no need for trailing slash
        url = BASE_URL,
        headers={
            "Authorization": f"Bearer {PROVISIONING_API_KEY}",
            "Content-Type": "application/json"
        },
        json={
            "name": name,   # Name of the API key
            "label": label,
            "limit": limit  # Optional credit limit
        }
    )
    # return the response
    if response.status_code == 201:
        print("API key created successfully.")
        return response.json()  # Return the JSON response containing the API key details
    else:
        print(f"Failed to create API key: {response.status_code} - {response.text}")
        return None
    
today = datetime.now().strftime("%Y_%m_%d")
# let's test the function to create a new API key
new_key = provision_open_router_keys(
    name=f"BSSDH 2025 Workshop {today}",
    label=f"BSSDH 2025 Workshop {today} key",
    limit=2
)

API key created successfully.


In [21]:
# let's try creating a budget key with 0.50 usd limit
new_budget_key = provision_open_router_keys(
    name="BSSDH 2025 Workshop Budget Key",
    label="BSSDH 2025 Workshop Budget Key",
    limit=0.50
)


API key created successfully.


In [12]:
# let's save keys to individual files in temp folder that is sibling the notebook folder
save_path = Path("../temp")
# note: I have excluded temp folder by adding temp entry in .gitignore file so it will not be committed to the repository
# otherwise everyone in the world would have access to your API keys!!!! if your repository is public
# create temp folder if it does not exist
save_path.mkdir(parents=True, exist_ok=True)
# save new_key to a file
if new_key:
    file_name = f"api_key_{today}.json"
    with open(save_path / file_name, "w") as f:
        json.dump(new_key, f, indent=4)
    print(f"New key saved to {save_path / file_name}")
# save new_budget_key to a file
# first check if new_budget_key variabl eexists at all
# chedk if variable actually exists
if 'new_budget_key' in locals() and new_budget_key:
    with open(save_path / "new_budget_key.json", "w") as f:
        json.dump(new_budget_key, f, indent=4)
    print(f"New budget key saved to {save_path / 'new_budget_key.json'}")

New key saved to ..\temp\api_key_2025_08_06.json


## Getting Previously created keys
We can use the OpenRouter API to get a list of previously created API keys. This is useful to check if the keys were created successfully and to see the limits and other details of the keys.

Again be very careful with retrieved keys, as they can be used to make API calls and drain your account's balance.

In [13]:
def get_keys(provision_key=open_router_bssdh_provisioner):
    # we use example from https://openrouter.ai/docs/features/provisioning-api-keys
    PROVISIONING_API_KEY = provision_key
    BASE_URL = "https://openrouter.ai/api/v1/keys"
    # List the most recent 100 API keys
    response = requests.get(
        BASE_URL,
        headers={
            "Authorization": f"Bearer {PROVISIONING_API_KEY}",
            "Content-Type": "application/json"
        }
    )

    # return the response
    if response.status_code == 200:
        print("API keys retrieved successfully.")
        return response.json()  # Return the JSON response containing the API keys
    else:
        print(f"Failed to retrieve API keys: {response.status_code} - {response.text}")
        return None
    
keys = get_keys()
# how many keys we have?
if keys:
    print(f"Number of keys retrieved: {len(keys)}")


API keys retrieved successfully.
Number of keys retrieved: 1


In [14]:
# print keys.keys()
# keys is a dictionary so it has keys ...
print("Keys available in the response:")
for key in keys.keys():
    print(f"key - {key}")
# data actually holds the list of keys but we will not print it here as it may contain sensitive information
# we will just see how many keys we have
if 'data' in keys:
    print(f"Number of API keys in data: {len(keys['data'])}")

Keys available in the response:
key - data
Number of API keys in data: 10


In [15]:
# let's get first key that contains Segmentation in its name
# in our particular case this key is already used so we are not afraid to print it by accident but generally be very careful with retrieved keys, 
# theoretically hash of the key can be used to identify the key without revealing it
if keys and 'data' in keys:
    segmentation_keys = [key for key in keys['data'] if 'Segmentation' in key['name']]
    if segmentation_keys:
        print(f"Found {len(segmentation_keys)} keys with 'Segmentation' in their name.")
        # print("First key with 'Segmentation':", segmentation_keys[0]) # prints hash only but still be careful with this
    else:
        print("No keys found with 'Segmentation' in their name.")

Found 1 keys with 'Segmentation' in their name.


## Creating keys for all participants

Now that we have all participants' data in a DataFrame and a function to create API keys, we can iterate over the DataFrame and create API keys for each participant.

In [ ]:
# let's iterate over all rows of the dataframe and create API keys for each participant
# we will use Name and Surname columns to create unique names and labels for the keys
# we will store response.json in a python list
# we will store the actual key which sits in the 'key' field of the response in the dataframe under key column

# all of this will be done through a function called provision_open_router_keys
# the parameters of the function will be dataframe, name_prefix = "BSSDH_2025", label_prefix = "BSSDH_2025_Key", limit = 1
import time
def provision_open_router_keys_for_participants(df, provision_key=open_router_bssdh_provisioner, 
                                                 name_prefix="BSSDH_2025", 
                                                 label_prefix="BSSDH_2025_Key", 
                                                 limit=1,
                                                 delay=0.1, # in case we want to add a delay between requests to avoid hitting the API rate limit
                                                 verbose=True
                                                 ):
    # create a list to store the keys
    keys_list = []
    # create df column called api_key
    df['api_key'] = None  # Initialize the 'api_key' column with None values
    
    # check if the DataFrame has 'Name' and 'Surname' columns
    if 'Name' not in df.columns or 'Surname' not in df.columns:
        raise ValueError("DataFrame must contain 'Name' and 'Surname' columns to provision keys.")
    if verbose:
        print(f"Provisioning keys for {len(df)} participants...")
    # iterate over the DataFrame rows
    for index, row in tqdm(df.iterrows()):
        # create unique name and label for the key
        name = f"{name_prefix}_{row['Name']}_{row['Surname']}"
        label = f"{label_prefix}_{row['Name']}_{row['Surname']}"
        # provision the key
        new_key = provision_open_router_keys(provision_key, name, label, limit)
        if new_key:
            # add the key to the list
            keys_list.append(new_key)
            # store the key in the DataFrame under 'key' column
            df.at[index, 'api_key'] = new_key.get('key', None)  # get 'key' field or None if it does not exist
        time.sleep(delay)  # add a delay to avoid hitting the API rate limit if any
    return keys_list

In [20]:
# now we can finally provision keys for all participants
keys_list = provision_open_router_keys_for_participants(df, # the rest are defaults so we could skip them actually
                                                        #    provision_key=open_router_bssdh_provisioner, 
                                                        #    name_prefix="BSSDH_2025_Workshop", 
                                                        #    label_prefix="BSSDH_2025_Workshop_Key", 
                                                        #    limit=1,
                                                        #    delay=0.1,  # 100 ms delay between requests
                                                        #    verbose=True
                                                           ) 
# how many keys we have provisioned?
if keys_list:
    print(f"Number of keys provisioned: {len(keys_list)}")   

Provisioning keys for 54 participants...


1it [00:00,  1.35it/s]

API key created successfully.


2it [00:01,  1.46it/s]

API key created successfully.


...

API key created successfully.


53it [00:27,  1.99it/s]

API key created successfully.


54it [00:27,  1.96it/s]

API key created successfully.
Number of keys provisioned: 54


## Saving key information

We want to save the created keys to a file so that we can use them later. We will save the keys in a JSON file in a temporary folder. The file will contain the key details, including the name, label, and limit.

Also we will save the modified DataFrame with the new column containing the API keys. This will allow us to easily access the keys later and to share them with the participants.

**Main thing is to make sure this information is not shared publicly, as it contains sensitive information that can be used to make API calls and drain your account's balance.**

In [21]:
# now let's save the keys list as a JSON file in the temp folder
# some of the participants names have non ASCII characters so we will use ensure_ascii=False to save them correctly
if keys_list:
    with open(save_path / "provisioned_keys.json", "w", encoding='utf-8') as f:
        json.dump(keys_list, f, indent=4, ensure_ascii=False)
    print(f"Provisioned keys saved to {save_path / 'provisioned_keys.json'}")

Provisioned keys saved to ..\temp\provisioned_keys.json


In [22]:
# let's save dataframe in BSSDH_2025_provisioned_keys.xlsx
df.to_excel(save_path / "BSSDH_2025_provisioned_keys.xlsx", index=False, engine='openpyxl')